In [1]:
import numpy as np 
import pandas as pd
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.pipeline import Pipeline
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
df = pd.read_csv ("train.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         60000 non-null  int64 
 1   comment    60000 non-null  object
 2   subreddit  60000 non-null  object
dtypes: int64(1), object(2)
memory usage: 937.6+ KB


In [4]:
def clean_text(df):
    all_reviews = list()
    lines = df["comment"].values.tolist()
    for text in lines:
        text = text.lower()
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        tokens = word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        stop_words.discard("not")
        PS = PorterStemmer()
#         words = [w for w in words if not w in stop_words]
        words = [PS.stem(w) for w in words if not w in stop_words]
        words = ' '.join(words)
        all_reviews.append(words)
    return all_reviews

all_reviews = clean_text(df)
all_reviews[0:2]

['think prestig point not expir ever skin buy avail set durat exempl year releas anoth skin vault old one make also limitededit skin also pleas love god nt rereleas skin need grind prestig shop would suck everyon grind',
 'what go happen refus asilum appeal']

In [5]:
c = all_reviews
filtered_sentence = [] 
freq_count_limit = FreqDist()
lemmatizer=WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

for i in c:
    comment_tokens = word_tokenize(i)
    for words in comment_tokens:
        if words not in stop_words: 
            filtered_sentence.append(words) 
        
            limit_words = lemmatizer.lemmatize(words)
#     for word in root_words:
            freq_count_limit[limit_words.lower()]+=1
freq_count_limit

FreqDist({'nt': 12955, 'like': 9662, 'get': 7121, 'one': 5971, 'peopl': 5884, 'would': 5745, 'think': 4914, 'go': 4786, 'time': 4431, 'make': 4394, ...})

In [6]:
freq_count_limit.pop("nt")

12955

In [7]:
# sorted_cleaned_words = sorted(freq_count_limit.items(), key=lambda x: x[1], reverse=True)
# len(sorted_cleaned_words)

In [8]:
# lists = sorted_cleaned_words
# vocabulary , count = zip(*lists)
# vocabulary  = list(vocabulary )

In [9]:
# top_100_words = vocabulary[:100]

In [10]:
#######################################################################

In [23]:
# pipe = Pipeline([('count', CountVectorizer(freq_count_limit=freq_count_limit)),
#                      ('tfid', TfidfTransformer())]).fit(df['subreddit'])

In [12]:
# pipe['count'].transform(all_reviews).toarray()

In [13]:
############################################################

In [14]:
# 1
# CV = CountVectorizer(min_df=3)   
# X = CV.fit_transform(all_reviews)
# y = CV.fit_transform(df["subreddit"])
# print(np.shape(X))
# print(np.shape(y))

In [15]:
# 1
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [16]:
# 1
# model_NB = GaussianNB()
# model_NB.fit(X_train,y_train)

# y_pred = model_NB.predict(X_test)

In [17]:
# 1
# print(accuracy_score(y_test, y_pred))
# print(f1_score(y_test, y_pred))
# print(precision_score(y_test, y_pred))